In [45]:
import sys
import pandas as pd
import os
from datetime import datetime
import numpy as np

In [46]:
# os.chdir('/home/tangosvc/code/01_sumin')
# os.getcwd()

In [42]:
# Cell별 Nbr Cell 정보
def getSiteInfo() :
    df = pd.read_csv('site_info.csv', encoding = 'utf-8', header = 1) 
    df = df[['SISUL Code', 'NBR(1st).1']]
    df.columns = ['SISUL_CD1', 'NBR_CD1']
    df = df.dropna(how ='any')
    return df

In [47]:
# Read Result File 
def getResult(seq, yymmdd) :

    resultPrefix = "TEOS_TO_SON_RL_RESULT_"

    nm = resultPrefix + str(yymmdd) + "_[" + str(seq) + "].dat"

    try:
        df = pd.read_csv(nm, sep = '|')
        print(nm)
        df.columns = ['REQUEST_DATE','SCENARIO_SEQ','REQUEST_SEQ','XPOS','YPOS',
                      'SISUL_CD1','RSRP1','SINR1',
                      'SISUL_CD2','RSRP2','SINR2',
                      'SISUL_CD3','RSRP3','SINR3']

    except FileNotFoundError:
        pass

    return df

# State Quality 계산

def getStateQuality(seq, yymmdd = None, wa = 0.5, wb = 0.5) :

    if yymmdd is None :
         yymmdd = datetime.today().strftime("%Y%m%d")

    path = '/home/tangosvc/code/data/' + yymmdd
    
    os.chdir(path)
    result = getResult(seq, yymmdd)
   
    # Cell SINR avg , 5%
    def q5(x):
        return x.quantile(0.05)

    f = {'SINR1': ['mean', q5], 'RSRP1': ['mean']}
    sqResult = result.groupby('SISUL_CD1').agg(f)
    sqResult['sq'] = sqResult[('SINR1', 'mean')] * wa + (1-wa) * sqResult[('SINR1', 'q5')]

    # Nbr Cell

    def getNbrSq(x) :
       
        os.chdir('/home/tangosvc/code/data')
        nbrSite = getSiteInfo()
        tmp = list(sqResult.index)

        nbrCell = list(nbrSite.NBR_CD1[nbrSite.SISUL_CD1 == x])

        if len(nbrCell) == 1 :
            nbrSq = sqResult.sq[tmp.index(nbrCell[0])]
        else :
            nbrSq = 99999 # nbrCell 없는 경우

        return nbrSq

    sqResult['nbrSq'] = list(map(lambda i: getNbrSq(i), sqResult.index.to_list()))

    # Global SQ
    # sqResult['gsq'] = sqResult['sq'] * wb + (1 - wb) * sqResult[('nbrSq')]
    sqResult['gsq'] = np.where(sqResult['nbrSq'] == 99999, sqResult['sq'] , sqResult['sq'] * wb + (1 - wb) * sqResult[('nbrSq')])

    sqResult.columns =  ['SINR1_Mean', 'SINR1_Q5', 'RSRP1_Mean', 'sq', 'nbrSq', 'gsq']
    sqResult['SISUL_CD'] = list(sqResult.index)

    return sqResult

In [48]:
# State Quality 계산 실행
dfSq = getStateQuality(12, yymmdd = '20191018', wa = 0.5, wb = 0.5) 
dfSq

TEOS_TO_SON_RL_RESULT_20191018_[12].dat


,SINR1_Mean,SINR1_Q5,RSRP1_Mean,sq,nbrSq,gsq,SISUL_CD
SISUL_CD1,,,,,,,
201930377,14.298912,3.9310,-82.890204,9.114956,8.325611,8.720283,201930377
201930926,15.073333,6.7730,-90.478889,10.923167,99999.000000,10.923167,201930926
201930993,8.534781,1.5705,-100.748377,5.052640,5.302971,5.177805,201930993
201931087,8.947853,4.8390,-98.515153,6.893426,99999.000000,6.893426,201931087
201932227,12.387031,4.2020,-90.292656,8.294516,99999.000000,8.294516,201932227
201932623,14.111148,4.2500,-94.323115,9.180574,99999.000000,9.180574,201932623
201933468,9.488657,3.5210,-98.474478,6.504828,99999.000000,6.504828,201933468
201933663,16.297000,-0.3525,-99.457250,7.972250,99999.000000,7.972250,201933663
201934295,12.633988,-0.4325,-90.017262,6.100744,99999.000000,6.100744,201934295


'/home/tangosvc/code/01_sumin'